HW6 Practical / Amin Ravanbakhsh 96109725

In [1]:
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cvxopt
from cvxopt import matrix, solvers

Loading Dataset

In [2]:
digits = datasets.load_digits()
digits
df = pd.DataFrame(data= np.c_[digits['data'], digits['target']], columns= digits['feature_names'] + ['target'])
df.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,...,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4.0


A) Splitting Dataset (At Least 15 Data Per Class In Test Data)

In [3]:
classes_count = 10

dataframes = [df[df["target"] == i] for i in range(classes_count)]

df_train = pd.DataFrame()
df_test = pd.DataFrame()

ninety_percent = int(df.shape[0] * 0.9)  #1617
ten_percent = df.shape[0] - ninety_percent  #180

for i in range(classes_count):
    temp = dataframes[i].sample(frac = 1)
    df_test = df_test.append(temp[:int(ten_percent / 10)])
    df_train = df_train.append(temp[int(ten_percent / 10):])
    

df_train = df_train.sample(frac = 1)
df_test = df_test.sample(frac = 1)

x_train = df_train.drop("target" , axis = 1)
y_train_s = df_train[["target"]]

x_test = df_test.drop("target" , axis = 1)
y_test_s = df_test[["target"]]

x_train_s = x_train / x_train.max().max()
x_test_s = x_test / x_test.max().max()

print("X Train Shape : " , x_train.shape)
print("Y Train Shape : " , y_train_s.shape)
print("X Test Shape : " , x_test.shape)
print("Y Test Shape : " , y_test_s.shape)

X Train Shape :  (1617, 64)
Y Train Shape :  (1617, 1)
X Test Shape :  (180, 64)
Y Test Shape :  (180, 1)


SVM One Against All Class And Kernels

In [4]:
class SVM:
    def __init__(self , target ,x ,y ,C ,sigma):
        
        self.target = target
        self.x = x
        self.y = y
        self.C = C
        self.sigma = sigma
        
    def fit(self):
        
        self.y = np.array([1.0 if i == self.target else -1.0 for i in self.y]).reshape(-1 , 1)    
        self.K = gaussian_kernel(self.x,self.x,self.sigma) if self.sigma != None else linear_kernel(self.x,self.x)
        
        a = self.x.shape[0]
        P = matrix(np.outer(self.y, self.y) * self.K)  
        q = matrix(-np.ones((a, 1)))    
        G = matrix(np.vstack((np.eye(a) * -1, np.eye(a))))  
        h = matrix(np.hstack((np.zeros(a), np.ones(a) * self.C)))
        A = matrix(self.y, (1, a), "d")
        b = matrix(np.zeros(1))
        solvers.options['show_progress'] = False
        sol = solvers.qp(P, q, G, h, A, b)
        alphas = np.array(sol['x'])     
        S = ((alphas > 1e-5) * (alphas < self.C)).flatten()      
        w = ((self.y * alphas).T @ self.x).reshape(-1,1)
        b = self.y[S] - np.dot(self.x[S], w)
    
        return w, b[0]
    
def linear_kernel(x, y):
    return np.dot(x, y.T)

def gaussian_kernel(x, y, sigma = 1):    
    y = np.dot(np.sum(np.power(x, 2), 1).reshape(x.shape[0], 1), np.ones((1, x.shape[0])))
    return np.exp(-(y + y.T - 2 * np.dot(x, x.T)) / (2 * sigma ** 2))

def train(classes , x, y, C, sigma):
    w , b = [] , []
    for i in range(classes):
        model = SVM(i, x, y, C, sigma)
        w_t , b_t = model.fit()
        w.append(w_t)
        b.append(b_t)
        
    return w, b

def predict(classes_count ,w , b , x):
    prediction = [None for i in x] 
    for i in range(classes_count):
        temp_pred = np.sign(np.dot(x , w[i]) + b[i])
        for j in range(len(temp_pred)):
            if prediction[j] == None:
                if temp_pred[j] == 1:
                    prediction[j] = i
                    
    return np.array(prediction)

B) Soft Margin SVM With Linear Kernel

Cross Validation

In [5]:
Kfold = 4
sets_length = int(len(x_train_s) / Kfold)

Cs = [0.01 , 0.1 , 1 , 10 , 100]

x_sets = []
y_sets = []

for i in range(Kfold):
    x_sets.append(x_train_s[sets_length * i:sets_length * (i + 1)])
    y_sets.append(y_train_s[sets_length * i:sets_length * (i + 1)])

models_accuracy = []
for C in Cs:
    test_accuracy = []
    for i in range(Kfold):
        x_test_set = x_sets[i]
        y_test_set = y_sets[i]
        x_train_set = x_sets[:i] + x_sets[i + 1:]
        y_train_set = y_sets[:i] + y_sets[i + 1:]
        x_train_set = pd.concat(x_train_set)
        y_train_set = pd.concat(y_train_set)
        w_valid , b_valid = train(classes_count , x_train_set.to_numpy(), y_train_set.to_numpy() , C , None)
        test_accuracy.append((predict(classes_count ,w_valid ,b_valid ,x_test_set.to_numpy()) 
                         == np.squeeze(y_test_set.to_numpy())).sum() / len(y_test_set))
    print("Accuracy With C =" , C ,": ", sum(test_accuracy) / len(test_accuracy))
    models_accuracy.append(sum(test_accuracy) / len(test_accuracy))   
    
print("All Accuracies : " , models_accuracy)
argmax = models_accuracy.index(max(models_accuracy))
print("Best C : " , Cs[argmax])
C_best = Cs[argmax]

Accuracy With C = 0.01 :  0.4783415841584158
Accuracy With C = 0.1 :  0.7976485148514851
Accuracy With C = 1 :  0.8366336633663366
Accuracy With C = 10 :  0.8787128712871287
Accuracy With C = 100 :  0.8997524752475248
All Accuracies :  [0.4783415841584158, 0.7976485148514851, 0.8366336633663366, 0.8787128712871287, 0.8997524752475248]
Best C :  100


In [6]:
w , b = train(classes_count , x_train_s.to_numpy(),y_train_s.to_numpy() , C_best , None)
w

[array([[ 0.00000000e+00],
        [-1.30325388e-01],
        [ 1.18184623e-01],
        [ 3.65710223e-01],
        [-4.76005573e-02],
        [-1.16683701e+00],
        [-1.21335401e+00],
        [-7.75031899e-02],
        [-3.63573213e-11],
        [-4.18859814e-02],
        [ 3.96360643e-02],
        [ 9.04666468e-01],
        [ 3.45918874e-01],
        [ 1.27419759e+00],
        [ 3.39998839e-01],
        [-9.30038280e-02],
        [-8.38021590e-11],
        [ 7.83566826e-01],
        [ 9.77758206e-01],
        [ 1.16969074e-01],
        [-1.58456726e+00],
        [ 1.09745807e+00],
        [-1.71909435e-01],
        [-1.06974886e-01],
        [-3.98986373e-11],
        [ 8.67017207e-02],
        [-1.69733371e-01],
        [-5.53873968e-01],
        [-2.23914952e+00],
        [ 8.40727568e-02],
        [ 3.67199088e-01],
        [-2.16524739e-11],
        [ 0.00000000e+00],
        [ 6.55469746e-01],
        [ 7.07460347e-01],
        [-1.54206842e-01],
        [-2.19542489e+00],
 

In [7]:
train_accuracy = (predict(classes_count ,w ,b ,x_train_s.to_numpy()) == np.squeeze(y_train_s.to_numpy())).sum() / len(y_train_s)
test_accuracy = (predict(classes_count ,w ,b ,x_test_s.to_numpy()) == np.squeeze(y_test_s.to_numpy())).sum() / len(y_test_s)

print("Linear Kernel Train Accuracy : " ,train_accuracy)
print("Linear Kernel Test Accuracy : " ,test_accuracy)

Linear Kernel Train Accuracy :  0.9616573902288188
Linear Kernel Test Accuracy :  0.8944444444444445


C) Soft Margin SVM With Gaussian Kernel

Cross Validation

In [8]:
sigma = [0.01 , 0.1 , 1 , 10 , 100]
models_accuracy = []

for S in sigma:
    test_accuracy = []
    for i in range(Kfold):
        x_test_set = x_sets[i]
        y_test_set = y_sets[i]
        x_train_set = x_sets[:i] + x_sets[i + 1:]
        y_train_set = y_sets[:i] + y_sets[i + 1:]
        x_train_set = pd.concat(x_train_set)
        y_train_set = pd.concat(y_train_set)
        
        w_valid , b_valid = train(classes_count , x_train_set.to_numpy(), y_train_set.to_numpy() , 100 , S)
        test_accuracy.append((predict(classes_count ,w_valid ,b_valid ,x_test_set.to_numpy()) 
                         == np.squeeze(y_test_set.to_numpy())).sum() / len(y_test_set))
    print("Accuracy With Sigma =" , S ,": ", sum(test_accuracy) / len(test_accuracy))
    models_accuracy.append(sum(test_accuracy) / len(test_accuracy)) 
    
print("All Accuracies : " , models_accuracy)
argmax = models_accuracy.index(max(models_accuracy))
print("Best Sigma : " , sigma[argmax])
sigma_best = sigma[argmax]

Accuracy With Sigma = 0.01 :  0.14789603960396042
Accuracy With Sigma = 0.1 :  0.14789603960396042
Accuracy With Sigma = 1 :  0.36448019801980197
Accuracy With Sigma = 10 :  0.8589108910891089
Accuracy With Sigma = 100 :  0.411509900990099
All Accuracies :  [0.14789603960396042, 0.14789603960396042, 0.36448019801980197, 0.8589108910891089, 0.411509900990099]
Best Sigma :  10


In [9]:
w , b = train(classes_count , x_train_s.to_numpy(),y_train_s.to_numpy() , C_best , sigma_best)
w

[array([[ 0.00000000e+00],
        [-2.81676437e+00],
        [ 3.23753016e+01],
        [ 3.81310890e+00],
        [ 1.46655725e+00],
        [-7.99048610e+01],
        [-4.75611107e+01],
        [-2.12729011e+00],
        [-9.00713275e-10],
        [-9.79757354e-01],
        [-1.71783336e+00],
        [ 5.65633874e+01],
        [ 5.82300860e+01],
        [ 8.91517730e+01],
        [ 1.89316406e+01],
        [-2.55274813e+00],
        [-9.79864368e-10],
        [ 4.62639104e+01],
        [ 4.33476784e+01],
        [-2.54773345e+00],
        [-9.66362201e+01],
        [ 8.00501627e+01],
        [-4.64489016e+00],
        [-4.31251121e+00],
        [-4.32465792e-10],
        [ 3.37547627e+01],
        [-5.69196504e-02],
        [-3.65333574e+01],
        [-1.57407777e+02],
        [ 3.89347086e+00],
        [ 2.23332302e+01],
        [-4.05727583e-10],
        [ 0.00000000e+00],
        [ 7.29611111e+01],
        [ 4.57757849e+01],
        [-3.69890825e+01],
        [-1.30220310e+02],
 

In [10]:
train_accuracy = (predict(classes_count ,w ,b ,x_train_s.to_numpy()) == np.squeeze(y_train_s.to_numpy())).sum() / len(y_train_s)
test_accuracy = (predict(classes_count ,w ,b ,x_test_s.to_numpy()) == np.squeeze(y_test_s.to_numpy())).sum() / len(y_test_s)

print("Gaussian Kernel Train Accuracy : " ,train_accuracy)
print("Gaussian Kernel Test Accuracy : " ,test_accuracy)

Gaussian Kernel Train Accuracy :  0.9053803339517625
Gaussian Kernel Test Accuracy :  0.8833333333333333
